# TV Algorithm

In [1]:
import numpy as np
from scipy.ndimage import gaussian_filter, convolve
from scipy.linalg import toeplitz
from scipy.signal import fftconvolve
from scipy.optimize import minimize


Degradation

In [2]:
def degrade_image(image, psf_size=5, noise_var=0.01):
    psf = np.ones((psf_size, psf_size)) / (psf_size ** 2)
    blurred = convolve(image, psf, mode='reflect')
    noise = np.random.normal(0, np.sqrt(noise_var), image.shape)
    return blurred + noise, psf


Convolution Matrix for Blur

In [3]:
def create_convolution_matrix(psf, shape):
    # Assumes circular boundary conditions
    from scipy.signal import fftconvolve
    kernel = np.pad(psf, [(0, shape[0] - psf.shape[0]), (0, shape[1] - psf.shape[1])], mode='constant')
    kernel = np.roll(kernel, -np.array(psf.shape) // 2, axis=(0, 1))
    return lambda x: fftconvolve(x, kernel, mode='same')

def apply_psf(image, psf):
    return fftconvolve(image, psf, mode='same')


Weight Matrix from TV Prior

In [4]:
def compute_weight_matrix(x):
    grad_x = np.gradient(x, axis=0)
    grad_y = np.gradient(x, axis=1)
    grad_mag = np.sqrt(grad_x**2 + grad_y**2)
    return 1 / (grad_mag + 1e-3)  # Avoid division by zero


Laplacian Matrix

In [5]:
def laplacian_matrix(shape):
    n = shape[0]
    L = -4 * np.eye(n) + np.eye(n, k=1) + np.eye(n, k=-1)
    return L


Regularized Solver

In [6]:
def solve_regularized(A, y, reg_param, laplacian):
    ATA = A.T @ A + reg_param * laplacian
    ATy = A.T @ y
    x = np.linalg.solve(ATA, ATy)
    return x


## TV1

In [7]:
def tv1_algorithm(y, psf, max_iter=50, reg_param=1e-2):
    # Initialization
    x = y.copy()  # Start with the degraded image as initial guess
    h = psf.copy()
    W = compute_weight_matrix(x)

    for i in range(max_iter):
        # Update x
        A = create_convolution_matrix(h, y.shape)
        L = np.eye(np.prod(y.shape))  # Simplified Laplacian for demonstration
        x_new = np.linalg.solve(A.T @ np.diag(W.ravel()) @ A + reg_param * L, A.T @ W.ravel() @ y.ravel())
        x = x_new.reshape(y.shape)

        # Update h (placeholder for gradient-based optimization)
        h = h  # In a full implementation, solve for h using optimization.

        # Update W
        W = compute_weight_matrix(x)

        # Check convergence
        if np.linalg.norm(x - x_new) < 1e-3:
            break

    return x, h


## TV2

In [8]:
def tv2_algorithm(y, psf, max_iter=50, reg_param=1e-2, gamma=1e-2):
    # Initialize variables
    x = y.copy()  # Start with degraded image
    h = psf.copy()
    laplacian = laplacian_matrix(y.shape)
    
    for i in range(max_iter):
        # Step 1: Update x
        A = lambda img: apply_psf(img, h)
        x_new = solve_regularized(A, y, reg_param, laplacian)
        
        # Step 2: Update h
        def blur_objective(h_flat):
            h_reshaped = h_flat.reshape(psf.shape)
            y_pred = apply_psf(x_new, h_reshaped)
            data_fidelity = np.sum((y - y_pred) ** 2)
            smoothness = gamma * np.sum(np.gradient(h_reshaped) ** 2)
            return data_fidelity + smoothness
        
        h_flat = minimize(blur_objective, h.ravel(), method='L-BFGS-B').x
        h = h_flat.reshape(psf.shape)
        
        # Step 3: Convergence check
        if np.linalg.norm(x - x_new) < 1e-3:
            break
        
        x = x_new

    return x, h
